In [ ]:
# !pip install seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


# House Price Prediction with Geospatial Features

**Goal**: Predict the sale price of a house using its attributes (size, rooms, age, location, etc.) and geospatial context (distance to downtown, schools, parks etc.).

Some key questions I aim to answer:
- *What are the top features influencing house prices in this region?*
- *Can we predict a home's market value within $\pm 10\%$ accuracy?*

## Stages to follow for the analysis:
(This will help me keep track)

1. Load the data
2. Clean/preprocess
3. Split train/validation/test
4. Train model
5. Evaluate model
6. Save model/predictions

## Data
- Source: [Ames Housing Data](https://www.kaggle.com/datasets/prevek18/ames-housing-dataset/data) from Kaggle
- Also using [this reference notebook](https://www.kaggle.com/code/lauvfpitipak/house-price-advanced-regression-xgboost-with-ridge) for analysis

In [1]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingRegressor

import xgboost as xgb

import os
import sys

In [2]:
df = pd.read_csv('../data/train.csv')
df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,2760,906392120,20,RL,11645,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,8,2006,New,Partial,294900
1,809,906226080,90,RL,7018,Pave,Reg,Bnk,AllPub,Inside,...,0,0,0,0,0,6,2009,WD,Alloca,153337
2,581,534127130,20,RL,11717,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2009,WD,Normal,185000
3,1144,531385060,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,190500
4,553,531477020,20,RH,8400,Pave,Reg,Lvl,AllPub,Inside,...,24,0,0,0,0,9,2009,WD,Normal,82000


In [3]:
df.describe()

,Order,PID,MS SubClass,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
count,2051.000000,2.051000e+03,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2032.000000,2050.000000,...,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000
mean,1471.054120,7.172680e+08,56.923452,10197.518284,6.068747,5.555339,1971.288152,1984.082886,100.843504,447.444878,...,93.768406,46.800098,23.947830,2.845929,15.054120,2.544125,55.010239,6.168211,2007.788396,180071.861043
std,842.460771,1.888624e+08,42.585277,8032.955701,1.425432,1.115406,30.113223,20.826108,180.779553,461.323101,...,126.300352,66.782384,66.518574,25.302379,54.100417,36.725446,577.304449,2.711400,1.312262,79680.819439
min,2.000000,5.263020e+08,20.000000,1470.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,13100.000000
25%,745.000000,5.284881e+08,20.000000,7484.000000,5.000000,5.000000,1954.000000,1965.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129500.000000
50%,1469.000000,5.354750e+08,50.000000,9503.000000,6.000000,5.000000,1973.000000,1993.000000,0.000000,376.000000,...,0.000000,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,160000.000000
75%,2205.500000,9.072002e+08,70.000000,11619.000000,7.000000,6.000000,2000.000000,2003.000000,161.000000,744.750000,...,168.000000,69.500000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,212000.000000
max,2930.000000,1.007100e+09,190.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1378.000000,5644.000000,...,1424.000000,742.000000,1012.000000,407.000000,576.000000,738.000000,17000.000000,12.000000,2010.000000,755000.000000
